# 05 - Differential Expression Analysis

**COVID-19 GSE171524 Single-Cell Analysis**

This notebook performs differential expression analysis between COVID and Control.

## Objectives
1. Global COVID vs Control DE analysis
2. Cell type-specific DE analysis
3. Pseudobulk DE with DESeq2-style approach
4. Pathway enrichment analysis
5. Compare with original paper findings

## Key Genes from Paper
- Myeloid: NEAT1, MALAT1 (up), AXL (down)
- Epithelial: KRT8, CLDN4, CDKN1A (DATP markers)
- Fibroblast: CTHRC1, POSTN, TNC (pFB markers)
- Cytokines: IL1B, IL6

In [ ]:
# Import libraries
import os
import sys
import warnings
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats, sparse
from statsmodels.stats.multitest import multipletests

warnings.filterwarnings('ignore')

# Add scripts to path
sys.path.insert(0, '../scripts')
from plotting import plot_volcano, COVID_COLORS
from utils import create_pseudobulk

# Settings
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, facecolor='white')

print(f"Scanpy: {sc.__version__}")

In [ ]:
# Define paths
INPUT_PATH = Path('../data/processed_data/adata_annotated.h5ad')
OUTPUT_DIR = Path('../results/tables')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FIGURE_DIR = Path('../results/figures/de')
FIGURE_DIR.mkdir(parents=True, exist_ok=True)

# Load annotated data
print(f"Loading: {INPUT_PATH}")
adata = sc.read_h5ad(INPUT_PATH)
print(f"Loaded: {adata.n_obs:,} cells, {adata.n_vars:,} genes")

In [ ]:
# Data overview
print("Cells by condition:")
print(adata.obs['condition'].value_counts())

print("\nCells by cell type:")
print(adata.obs['cell_type'].value_counts())

## Global DE: COVID vs Control (All Cells)

In [ ]:
# Run Wilcoxon rank-sum test
sc.tl.rank_genes_groups(
    adata,
    groupby='condition',
    groups=['COVID'],
    reference='Control',
    method='wilcoxon',
    n_genes=adata.n_vars
)

print("Computed DE: COVID vs Control (all cells)")

In [ ]:
# Extract results
de_global = sc.get.rank_genes_groups_df(adata, group='COVID')
de_global = de_global.rename(columns={'names': 'gene', 'logfoldchanges': 'log2FC', 'pvals_adj': 'padj'})

# Summary
sig_up = ((de_global['log2FC'] > 0.5) & (de_global['padj'] < 0.05)).sum()
sig_down = ((de_global['log2FC'] < -0.5) & (de_global['padj'] < 0.05)).sum()

print(f"Significant DEGs (|log2FC| > 0.5, padj < 0.05):")
print(f"  Upregulated in COVID: {sig_up}")
print(f"  Downregulated in COVID: {sig_down}")

# Top genes
print("\nTop 10 upregulated:")
print(de_global[de_global['log2FC'] > 0].head(10)[['gene', 'log2FC', 'padj']])

print("\nTop 10 downregulated:")
print(de_global[de_global['log2FC'] < 0].head(10)[['gene', 'log2FC', 'padj']])

In [ ]:
# Volcano plot
fig, ax = plt.subplots(figsize=(10, 8))

# Classify significance
de_global['significance'] = 'NS'
de_global.loc[(de_global['log2FC'] > 0.5) & (de_global['padj'] < 0.05), 'significance'] = 'Up'
de_global.loc[(de_global['log2FC'] < -0.5) & (de_global['padj'] < 0.05), 'significance'] = 'Down'

# Plot
colors = {'Up': '#E74C3C', 'Down': '#3498DB', 'NS': '#AAAAAA'}
for sig, color in colors.items():
    mask = de_global['significance'] == sig
    ax.scatter(
        de_global.loc[mask, 'log2FC'],
        -np.log10(de_global.loc[mask, 'padj'].clip(lower=1e-300)),
        c=color, alpha=0.5, s=10, label=f'{sig} ({mask.sum()})'
    )

# Threshold lines
ax.axhline(-np.log10(0.05), ls='--', c='gray', alpha=0.5)
ax.axvline(0.5, ls='--', c='gray', alpha=0.5)
ax.axvline(-0.5, ls='--', c='gray', alpha=0.5)

# Label top genes
top_up = de_global[de_global['significance'] == 'Up'].nlargest(10, 'log2FC')
top_down = de_global[de_global['significance'] == 'Down'].nsmallest(10, 'log2FC')

for _, row in pd.concat([top_up, top_down]).iterrows():
    ax.annotate(
        row['gene'],
        (row['log2FC'], -np.log10(max(row['padj'], 1e-300))),
        fontsize=8, alpha=0.8
    )

ax.set_xlabel('Log2 Fold Change (COVID vs Control)')
ax.set_ylabel('-Log10 Adjusted P-value')
ax.set_title('Differential Expression: COVID vs Control (All Cells)')
ax.legend()

plt.tight_layout()
plt.savefig(FIGURE_DIR / 'volcano_global.png', dpi=150)
plt.show()

In [ ]:
# Check paper-relevant genes
paper_genes = [
    'NEAT1', 'MALAT1', 'AXL',  # Myeloid
    'KRT8', 'CLDN4', 'CDKN1A',  # DATP
    'CTHRC1', 'POSTN', 'TNC',  # pFB
    'IL1B', 'IL6', 'TNF',  # Cytokines
    'AGER', 'SFTPC'  # AT1/AT2
]

print("Paper-relevant genes:")
paper_de = de_global[de_global['gene'].isin(paper_genes)].copy()
paper_de = paper_de.set_index('gene').loc[[g for g in paper_genes if g in paper_de['gene'].values]]
print(paper_de[['log2FC', 'padj', 'significance']])

## Cell Type-Specific DE Analysis

In [ ]:
# DE for each cell type
cell_types = adata.obs['cell_type'].unique()
de_results = {}

for ct in cell_types:
    print(f"\nProcessing: {ct}")
    adata_ct = adata[adata.obs['cell_type'] == ct].copy()
    
    # Check sample sizes
    n_covid = (adata_ct.obs['condition'] == 'COVID').sum()
    n_ctrl = (adata_ct.obs['condition'] == 'Control').sum()
    print(f"  COVID: {n_covid}, Control: {n_ctrl}")
    
    if n_covid < 50 or n_ctrl < 50:
        print(f"  Skipping: insufficient cells")
        continue
    
    # Run DE
    sc.tl.rank_genes_groups(
        adata_ct,
        groupby='condition',
        groups=['COVID'],
        reference='Control',
        method='wilcoxon',
        n_genes=adata_ct.n_vars
    )
    
    # Extract results
    de_ct = sc.get.rank_genes_groups_df(adata_ct, group='COVID')
    de_ct = de_ct.rename(columns={'names': 'gene', 'logfoldchanges': 'log2FC', 'pvals_adj': 'padj'})
    de_ct['cell_type'] = ct
    
    de_results[ct] = de_ct
    
    # Summary
    sig = ((de_ct['log2FC'].abs() > 0.5) & (de_ct['padj'] < 0.05)).sum()
    print(f"  Significant DEGs: {sig}")

print(f"\nCompleted DE for {len(de_results)} cell types")

In [ ]:
# Combine all cell type DE results
de_all_ct = pd.concat(de_results.values(), ignore_index=True)

# Count DEGs per cell type
deg_counts = []
for ct, df in de_results.items():
    sig = (df['log2FC'].abs() > 0.5) & (df['padj'] < 0.05)
    deg_counts.append({'cell_type': ct, 'direction': 'up', 'count': (sig & (df['log2FC'] > 0)).sum()})
    deg_counts.append({'cell_type': ct, 'direction': 'down', 'count': (sig & (df['log2FC'] < 0)).sum()})

deg_counts = pd.DataFrame(deg_counts)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(
    data=deg_counts,
    x='cell_type',
    y='count',
    hue='direction',
    palette={'up': '#E74C3C', 'down': '#3498DB'},
    ax=ax
)
ax.tick_params(axis='x', rotation=45)
ax.set_xlabel('Cell Type')
ax.set_ylabel('Number of DEGs')
ax.set_title('Differentially Expressed Genes per Cell Type')
ax.legend(title='Direction')

plt.tight_layout()
plt.savefig(FIGURE_DIR / 'deg_counts_by_celltype.png', dpi=150)
plt.show()

In [ ]:
# Volcano plots per cell type
n_cts = len(de_results)
ncols = 3
nrows = int(np.ceil(n_cts / ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(15, 4*nrows))
axes = axes.flatten()

for i, (ct, de_ct) in enumerate(de_results.items()):
    ax = axes[i]
    
    # Classify
    de_ct['sig'] = 'NS'
    de_ct.loc[(de_ct['log2FC'] > 0.5) & (de_ct['padj'] < 0.05), 'sig'] = 'Up'
    de_ct.loc[(de_ct['log2FC'] < -0.5) & (de_ct['padj'] < 0.05), 'sig'] = 'Down'
    
    colors = {'Up': '#E74C3C', 'Down': '#3498DB', 'NS': '#AAAAAA'}
    for sig, color in colors.items():
        mask = de_ct['sig'] == sig
        ax.scatter(
            de_ct.loc[mask, 'log2FC'],
            -np.log10(de_ct.loc[mask, 'padj'].clip(lower=1e-300)),
            c=color, alpha=0.5, s=5
        )
    
    ax.set_title(f'{ct}\n(Up:{(de_ct["sig"]=="Up").sum()}, Down:{(de_ct["sig"]=="Down").sum()})')
    ax.set_xlabel('Log2FC')
    ax.set_ylabel('-log10(padj)')

for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.savefig(FIGURE_DIR / 'volcano_by_celltype.png', dpi=150)
plt.show()

## Pseudobulk DE Analysis

In [ ]:
# Create pseudobulk per sample
# Sum raw counts per sample

if 'counts' in adata.layers:
    X = adata.layers['counts']
else:
    X = adata.X

if sparse.issparse(X):
    X = X.toarray()

# Create dataframe
df_counts = pd.DataFrame(X, index=adata.obs_names, columns=adata.var_names)
df_counts['sample_id'] = adata.obs['sample_id'].values

# Aggregate
pseudobulk = df_counts.groupby('sample_id').sum()

# Add condition
sample_condition = adata.obs.groupby('sample_id')['condition'].first()

print(f"Pseudobulk matrix: {pseudobulk.shape}")
print(f"\nSamples per condition:")
print(sample_condition.value_counts())

In [ ]:
# Simple pseudobulk DE using t-test
# For proper analysis, use DESeq2 or edgeR in R

covid_samples = sample_condition[sample_condition == 'COVID'].index
ctrl_samples = sample_condition[sample_condition == 'Control'].index

# Log-normalize pseudobulk
pseudobulk_norm = np.log1p(pseudobulk / pseudobulk.sum(axis=1).values[:, None] * 1e6)

# T-test for each gene
de_pseudo = []
for gene in pseudobulk.columns:
    covid_vals = pseudobulk_norm.loc[covid_samples, gene]
    ctrl_vals = pseudobulk_norm.loc[ctrl_samples, gene]
    
    # Skip low-expressed genes
    if covid_vals.mean() < 0.1 and ctrl_vals.mean() < 0.1:
        continue
    
    stat, pval = stats.ttest_ind(covid_vals, ctrl_vals)
    log2fc = covid_vals.mean() - ctrl_vals.mean()
    
    de_pseudo.append({
        'gene': gene,
        'log2FC': log2fc,
        'pval': pval,
        'covid_mean': covid_vals.mean(),
        'ctrl_mean': ctrl_vals.mean()
    })

de_pseudo = pd.DataFrame(de_pseudo)

# Multiple testing correction
de_pseudo['padj'] = multipletests(de_pseudo['pval'], method='fdr_bh')[1]

# Sort by significance
de_pseudo = de_pseudo.sort_values('pval')

print("Pseudobulk DE (t-test):")
sig_pseudo = ((de_pseudo['log2FC'].abs() > 0.5) & (de_pseudo['padj'] < 0.05)).sum()
print(f"Significant DEGs: {sig_pseudo}")

print("\nTop 20:")
print(de_pseudo.head(20)[['gene', 'log2FC', 'padj']])

## Pathway Enrichment

In [ ]:
# Try to use decoupler for pathway enrichment
try:
    import decoupler as dc
    
    # Get gene sets (MSigDB Hallmark)
    msigdb = dc.get_resource('MSigDB')
    hallmark = msigdb[msigdb['collection'] == 'hallmark']
    
    # Run enrichment on upregulated genes
    up_genes = de_global[(de_global['log2FC'] > 0.5) & (de_global['padj'] < 0.05)]['gene'].tolist()
    
    print(f"Testing {len(up_genes)} upregulated genes")
    
    # Over-representation analysis
    enr_up = dc.get_ora_df(
        df=de_global.set_index('gene'),
        net=hallmark,
        source='geneset',
        target='genesymbol'
    )
    
    print("\nTop enriched pathways (upregulated):")
    print(enr_up.head(10))
    
    DECOUPLER_AVAILABLE = True
    
except ImportError:
    print("decoupler not available, skipping pathway enrichment")
    print("Install with: pip install decoupler")
    DECOUPLER_AVAILABLE = False

In [ ]:
# Alternative: Simple GO enrichment using gseapy
try:
    import gseapy as gp
    
    # Get upregulated genes
    up_genes = de_global[(de_global['log2FC'] > 0.5) & (de_global['padj'] < 0.05)]['gene'].tolist()
    down_genes = de_global[(de_global['log2FC'] < -0.5) & (de_global['padj'] < 0.05)]['gene'].tolist()
    
    # Run enrichr on upregulated genes
    if len(up_genes) > 10:
        enr_up = gp.enrichr(
            gene_list=up_genes[:500],  # Limit for API
            gene_sets=['GO_Biological_Process_2023', 'KEGG_2021_Human'],
            organism='human',
            outdir=None
        )
        
        print("Top GO terms (upregulated in COVID):")
        print(enr_up.results[['Term', 'P-value', 'Adjusted P-value']].head(10))
        
        GSEAPY_AVAILABLE = True
    else:
        print("Not enough upregulated genes for enrichment")
        GSEAPY_AVAILABLE = False
        
except ImportError:
    print("gseapy not available, skipping GO enrichment")
    print("Install with: pip install gseapy")
    GSEAPY_AVAILABLE = False
except Exception as e:
    print(f"Enrichment error: {e}")
    GSEAPY_AVAILABLE = False

## Save Results

In [ ]:
# Save global DE results
de_global.to_csv(OUTPUT_DIR / 'de_global_covid_vs_control.csv', index=False)
print(f"Saved: {OUTPUT_DIR / 'de_global_covid_vs_control.csv'}")

# Save cell type-specific DE
de_all_ct.to_csv(OUTPUT_DIR / 'de_by_celltype.csv', index=False)
print(f"Saved: {OUTPUT_DIR / 'de_by_celltype.csv'}")

# Save pseudobulk DE
de_pseudo.to_csv(OUTPUT_DIR / 'de_pseudobulk.csv', index=False)
print(f"Saved: {OUTPUT_DIR / 'de_pseudobulk.csv'}")

In [ ]:
# Create summary table of key genes
key_genes = [
    # Myeloid dysfunction
    'NEAT1', 'MALAT1', 'AXL', 'MERTK',
    # DATP markers
    'KRT8', 'CLDN4', 'CDKN1A', 'KRT17',
    # Pathological fibroblasts
    'CTHRC1', 'POSTN', 'TNC', 'COL1A1',
    # Cytokines
    'IL1B', 'IL6', 'TNF', 'CCL2', 'CXCL8',
    # AT1/AT2
    'AGER', 'SFTPC', 'SFTPA1',
    # T cell markers
    'PDCD1', 'LAG3', 'TIGIT', 'GZMB',
    # Interferon
    'ISG15', 'MX1', 'IFIT1'
]

key_de = de_global[de_global['gene'].isin(key_genes)].copy()
key_de = key_de.set_index('gene')
key_de = key_de.loc[[g for g in key_genes if g in key_de.index]]

print("Key COVID-related genes:")
print(key_de[['log2FC', 'padj', 'significance']].to_string())

key_de.to_csv(OUTPUT_DIR / 'de_key_genes.csv')

## Comparison with Paper Findings

### Expected from Melms et al. 2021:

| Gene | Expected | Cell Type |
|------|----------|------------|
| NEAT1, MALAT1 | Up | Myeloid |
| AXL | Down | Myeloid |
| KRT8, CLDN4, CDKN1A | Up | Epithelial (DATP) |
| CTHRC1 | Up | Fibroblast (pFB) |
| IL1B | Up | Myeloid |
| IL6 | Up | Epithelial |

In [ ]:
# Heatmap of key genes by cell type
key_available = [g for g in key_genes if g in adata.var_names]

# Use log-normalized data
expr_data = []
for ct in adata.obs['cell_type'].unique():
    for cond in ['Control', 'COVID']:
        mask = (adata.obs['cell_type'] == ct) & (adata.obs['condition'] == cond)
        if mask.sum() > 0:
            X_subset = adata[mask, key_available].X
            if sparse.issparse(X_subset):
                mean_expr = np.asarray(X_subset.mean(axis=0)).flatten()
            else:
                mean_expr = X_subset.mean(axis=0).flatten()
            for gene, expr in zip(key_available, mean_expr):
                expr_data.append({
                    'cell_type': ct,
                    'condition': cond,
                    'gene': gene,
                    'expression': float(expr)
                })

expr_df = pd.DataFrame(expr_data)
expr_pivot = expr_df.pivot_table(
    index=['cell_type', 'condition'],
    columns='gene',
    values='expression'
)

# Plot heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(
    expr_pivot[key_available],
    cmap='viridis',
    center=0,
    annot=False
)
plt.title('Key Gene Expression by Cell Type and Condition')
plt.tight_layout()
plt.savefig(FIGURE_DIR / 'heatmap_key_genes.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary

### Global DE Analysis
- Identified DEGs between COVID and Control across all cells
- Key paper genes validated

### Cell Type-Specific DE
- Performed DE within each major cell type
- Different cell types show distinct COVID responses

### Pseudobulk Analysis
- Sample-level DE to account for patient variability

### Output Files
- `de_global_covid_vs_control.csv` - All cells DE
- `de_by_celltype.csv` - Cell type-specific DE
- `de_pseudobulk.csv` - Pseudobulk DE
- `de_key_genes.csv` - Summary of key genes

### Next Steps
→ **06_myeloid_analysis.ipynb**: Deep dive into myeloid dysfunction

In [ ]:
# Session info
print("\n=== Session Info ===")
print(f"Scanpy: {sc.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")